## GAP Data Analytics, Data Retrieval

This Jupyter Notebook allows for automation in the process of extracting GitHub statistics relevant for the redistribution of the GAP programming language. To extract data from the PyGithub API, it is first necessary to install the PyGitHub library. This library provides a Python wrapper for the GitHub REST API.

In [ ]:
# Import sys module for various system-specific parameters and functions
# Exclude lines that are already satisfied using the grep search command
import sys
!{sys.executable} -m pip install numpy pandas matplotlib seaborn PyGithub | grep -v 'already satisfied'

# Import required libraries and packages
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from ydata_profiling import ProfileReport

# Import modules from other project scripts
from constants import *

### Managing GitHub API Connection

Connecting to GitHub and verifying the user GitHub token is done through storing the access token as an environment variable. This way, the access token is not exposed in the script. The function for getting the token is imported from the utils file in the project. The API has a call limit of 5000 calls per hour, which creates the need to track the usage and remaining calls.

In [ ]:
# Track the rate limit for GitHub compared to calls used, and see when the limit will reset
remaining_requests, request_limit = g.rate_limiting
print(f"Request Limit for API Calls: {request_limit}")
print(f"Remaining Requests for API Calls: {remaining_requests}")

limit_reset_time = g.rate_limiting_resettime
reset_time = datetime.fromtimestamp(limit_reset_time).strftime('%Y-%m-%d %H:%M:%S')
print(f"Reset Time for API Calls: {reset_time}")

### General Statistics on GAP Packages and Distribution

Core statistical metrics based on the general state of packages, relevant for the management of GAP from GitHub, are provided below. These numbers are helpful in providing some foundational understanding of the current sitation of the programming language, in terms of development, distribution and redistribution.

In [ ]:
# Get the number of repositories that are public for gap-packages organisation on GitHub
org = g.get_organization(ORG_NAME_PACKAGES)
repos = org.get_repos(type="public")
total_packages = repos.totalCount
print(f"Number of GAP packages fra GAP Respository: {total_packages}")

In [ ]:
# Number of GAP packages hosted elsewhere on GitHub
# The information is attempted gathered through the web scraping technique provided by Beautiful Soup
# NB: These numbers are only indicative and not completely accurate due to the webpage listing style, counts per parent list item
url = "https://gap-packages.github.io/"
response = requests.get(url)

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.text, "html.parser")

# Find the section of the webpage with the packages stored elsewhere on GitHub
section = soup.find("section", id="main-content")
heading = section.find(id="packages-hosted-elsewhere-on-github")
ul = heading.find_next("ul")

# Do not include any child elements that are ul or li, as not to let these increase the count
packages = ul.find_all("li", recursive=False)
count = len(packages)

print(f"Number of GAP entities or packages hosted elsewhere on GitHub: {count}")

### Individual Statistics per GitHub GAP Package

Individual statistical metrics given per GAP repository managed by the gap-packages organisation on GitHub, are provided below. These numbers are helpful in providing some insight to aid decision-making when deciding how to address the respective packages, in terms of their status on need as well as readiness for redistribution. The process is divided into two parts, by first defining functions to get the data, and then retrieving it for each repository.

##### Functions to Retrieve Individual Statistics

In [ ]:
# Function to get the number of releases for a repository
# Also provide the latest release date, to indicate whether a package was released relatively recently
def get_total_releases(repo):
    releases = repo.get_releases()
    total_releases = releases.totalCount

    if total_releases > 0:
        latest_release = releases[0]
        latest_release_date = latest_release.published_at.date()
        return repo.name, total_releases, latest_release_date
    else:
        return repo.name, total_releases, None

In [ ]:
# Function to get the age for a respository, measured in days
def get_repository_age(repo):
    age = (datetime.now().date() - repo.created_at.date())
    return repo.name, age.days

In [ ]:
# Function to get the last activity event for a repository
# Watch event is excluded, as not to hide other events that would be of greater significance
# NB: Only events within the past 90 days are included in the search, per API limitations
def get_last_event(repo):
    # Define dictionary for events to be considered
    EVENT_TYPES = {
    "CommitCommentEvent": "Comment was made on a commit",
    "CreateEvent": "New branch or tag in repository",
    "DeleteEvent": "Branch or tag was deleted from the repository",
    "ForkEvent": "Repository was forked",
    "IssueEvent": "An issue was opened, closed or edited",
    "IssueCommentEvent": "Comment made on an issue",
    "PullRequestEvent": "Pull request was opened, closed, merged or synchronised",
    "PullRequestReviewEvent": "Pull request review was submitted",
    "PullRequestReviewCommentEvent": "Comment was made on a pull request review",
    "PushEvent": "Push to the repository",
    "ReleaseEvent": "Release was published for the repository",
    }

    events = repo.get_events()
    if events.totalCount > 0:
        last_event = events[0]
        last_event_type = EVENT_TYPES.get(last_event.type)
        last_event_time = last_event.created_at.date()
        return repo.name, last_event_time, last_event_type
    else:
        return repo.name, None, None

##### Get and Display Statistics for Each Individual GAP Package

In [ ]:
# Display alternative 1: Printing out information
# Generate relevant statistics for all repositories managed by the gap-packages organisation on GitHub
for repo in repos:
    # Call function for total releases for the repository
    repo_name, total_releases, latest_release_date = get_total_releases(repo)
    if total_releases > 0:
        print(f"Total Releases for {repo_name}: {total_releases}")
        print(f"Latest Release Date: {latest_release_date}")
    else:
        print(f"No releases for {repo_name}")
    
    # Call function for total releases
    repo_name, repo_age = get_repository_age(repo)
    print(f"Repository: {repo_name}, Age: {repo_age} days")

    # Call function for last event type for the repository
    repo_name, last_event_time, last_event_type = get_last_event(repo)
    if last_event_time is not None:
        print(f"Last Activity Time for {repo_name}: {last_event_time}")
        print(f"Last Activity Type for {repo_name}: {last_event_type}")
    else:
        print(f"No activity in {repo_name} within the past 90 days")

In [ ]:
# Display alternative 2: Using a Pandas DataFrame for more user-friendly formatting
# Using Python throwaway variable (_) where the actual return value will not be used
repo_stats = []

for repo in repos:
    repo_dict = {}
    repo_dict['Name'] = repo.name

    # Get and display total releases
    _, total_releases, latest_release_date = get_total_releases(repo)
    repo_dict['Total Releases'] = total_releases
    repo_dict['Latest Release Date'] = latest_release_date

    # Get and display repository age
    _, repo_age = get_repository_age(repo)
    repo_dict['Age'] = repo_age

    # Get and display last event
    _, last_event_time, last_event_type = get_last_event(repo)
    repo_dict['Last Activity Time'] = last_event_time
    repo_dict['Last Activity Type'] = last_event_type

    # Add the current repo stats to the list of repo stats
    repo_stats.append(repo_dict)

df = pd.DataFrame(repo_stats)
df = df.sort_values(by='Age', ascending=False)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print(df)

In [ ]:
# Display alternative 3: Creating a ProfileReport for more statistical analysis
# Use profiling library to see other, generalised statistics
profile = ProfileReport(df, title="Statistics for packages by gap-packages on GitHub")
profile.to_widgets()